## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated, multi-agent systems:

**LangChain**: A versatile framework for building applications powered by large language models (LLMs), LangChain offers modular components for complex workflows, context management, and multi-step tasks. It integrates seamlessly with various LLMs and external tools, making it suitable for conversational AI, research tools, and document analysis. ([curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai))

**LangGraph**: An extension of LangChain, LangGraph focuses on stateful, multi-agent applications, providing tools for agent coordination, visual graph-based workflows, and advanced error handling. It's ideal for applications requiring dependency management and logical flow across multi-step tasks, such as storytelling, multi-step chatbots, and strategic planning tools. ([curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai))

**CrewAI**: Designed for orchestrating role-playing AI agents, CrewAI enables the creation of specialized agents that collaborate on complex tasks. It features role-based agent architecture, dynamic task planning, and inter-agent communication protocols, making it well-suited for virtual product development teams and complex, multi-step workflows. ([chatbase.co](https://www.chatbase.co/blog/ai-agent-frameworks?utm_source=openai))

These frameworks are recognized for their flexibility, scalability, and active community support, facilitating the development of advanced AI agent systems across various domains. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [5]:

message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='Identify currently leading AI agent frameworks as of 2025', query='most popular AI agent frameworks 2025'), WebSearchItem(reason='Find success metrics and user adoption statistics for AI agent frameworks in 2025', query='successful AI agent frameworks user adoption 2025'), WebSearchItem(reason='Review expert analysis and comparisons of AI agent frameworks in 2025', query='best AI agent frameworks expert reviews 2025'), WebSearchItem(reason='Explore recent innovations and updates in AI agent frameworks by May 2025', query='latest AI agent framework innovations May 2025'), WebSearchItem(reason='Check developer forums and communities for popular AI agent frameworks discussion 2025', query='AI agent frameworks developer community popularity 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [6]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [12]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [13]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x112e9f9c0>, strict_json_schema=True)

In [14]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [15]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [16]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [17]:
query ="What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# AI Agent Frameworks: A Comprehensive Assessment (May 2025)

## Table of Contents

1. [Introduction](#introduction)  
2. [Methodology](#methodology)  
3. [Leading AI Agent Frameworks](#leading-ai-agent-frameworks)  
     3.1. [LangChain](#langchain)  
     3.2. [LangGraph](#langgraph)  
     3.3. [CrewAI](#crewai)  
     3.4. [Microsoft's AutoGen](#microsofts-autogen)  
     3.5. [AI2Agent](#ai2agent)  
     3.6. [Other Notable Frameworks](#other-notable-frameworks)  
4. [Use Cases and Applications](#use-cases-and-applications)  
5. [Challenges and Considerations](#challenges-and-considerations)  
6. [Future Trends](#future-trends)  
7. [Conclusion](#conclusion)  
8. [References](#references)  

## Introduction  
The rapid evolution of artificial intelligence (AI) has led to the emergence of advanced frameworks for developing AI agents capable of sophisticated tasks. By May 2025, many frameworks have proven their efficiency and popularity in diverse applications, underscoring their strategic significance in enterprise settings. This report delves into the leading AI agent frameworks dominating the landscape in May 2025, highlighting their unique features and applicability across various use cases.

## Methodology  
This report synthesizes insights from multiple reputable sources, including industry analyses, developer surveys, and expert commentary, to provide a cohesive overview of the AI agent frameworks currently shaping the market. A review of documented experiences from developers and organizations aids in understanding practical applications and assessing framework performance.

## Leading AI Agent Frameworks  
As of May 2025, the following frameworks are recognized for their capabilities and adoption across various sectors:

### LangChain  
LangChain is a versatile open-source framework designed for building applications powered by large language models (LLMs). It simplifies complex workflows by providing tools for context management, workflow creation, and integration with multiple LLMs such as OpenAI and Hugging Face. It is particularly well-suited for applications in conversational AI, research tools, and document analysis. However, it has been noted that LangChain can be somewhat complex for newcomers, requiring a steep learning curve to master its many features.  

**Key Features:**  
- Modular development capabilities  
- Integration with leading LLMs  
- Context management tools  

### LangGraph  
An extension of LangChain, LangGraph focuses on multi-agent systems that can collaborate and adapt to different tasks. It enhances LangChain’s functionalities by providing coordination tools for multiple agents, a visual graph-based workflow, and robust error-handling capabilities. LangGraph is typically utilized for applications such as storytelling, multi-step chatbots, and strategic planning tools.  

**Key Features:**  
- Support for stateful multi-agent applications  
- Advanced error handling  
- Visual workflow design  

### CrewAI  
CrewAI is a framework aimed at orchestrating role-based AI agents, allowing developers to create teams of specialized agents to tackle complex tasks. It features a role-based architecture which facilitates dynamic task planning and enhances inter-agent communication. CrewAI is particularly adept at managing intricate, multi-step workflows. Future updates are expected to introduce more sophisticated orchestration strategies.  

**Key Features:**  
- Role-based agent assignments  
- Inter-agent communication protocols  
- Dynamic task management  

### Microsoft’s AutoGen  
AutoGen is a multi-agent conversational system designed to support both autonomous and human-in-the-loop workflows. It features asynchronous messaging capabilities and event-driven interactions, making it suitable for tasks that require real-time concurrency and dynamic dialogue management. However, there are reported limitations in its applicability for specific tasks like compiling C source code and extracting data from complex files.  

**Key Features:**  
- Asynchronous messaging  
- Human-in-the-loop support  
- Event-driven interaction capabilities  

### AI2Agent  
AI2Agent is aimed at automating the deployment of AI projects, allowing for guideline-driven execution and self-adaptive debugging. Experiments indicate that AI2Agent can significantly reduce deployment times while improving success rates across various AI deployment cases. This framework is particularly valuable for teams looking to streamline their project initiation and execution phases in AI deployments.  

**Key Features:**  
- Automation of deployment processes  
- Self-adaptive debugging mechanisms  
- Enhanced project execution efficiency  

### Other Notable Frameworks  
- **OpenAI Swarm**: Focuses on lightweight and modular multi-agent systems designed for autonomous collaboration.  
- **Hugging Face Transformers Agents**: Facilitates the development, testing, and deployment of agents handling complex natural language tasks, leveraging advanced machine learning models.
- **Rasa**: An open-source solution ideal for creating customized conversational AI systems for industry-specific applications.
- **JADE**: Tailored for distributed systems, especially in IoT environments, providing robust communication protocols for complex system management.

## Use Cases and Applications  
The versatility of these frameworks enables their application across numerous industries and scenarios.  
- **Customer Support**: Automated agents handle inquiries and provide support without human intervention.  
- **Content Creation**: Frameworks like LangChain assist in generating insights and drafting documents.  
- **Project Management**: CrewAI allows project teams to collaborate effectively by assigning roles and managing tasks dynamically.  
- **Research Tools**: Leveraging LLMs, automated tools streamline the research process, enhancing accuracy and reducing time expenditure.  

## Challenges and Considerations  
While the frameworks offer substantial benefits, several challenges warrant attention:
- **Complexity for Beginners**: Many frameworks, particularly LangChain, may pose a steep learning curve for novice users.
- **Deployment Constraints**: Certain frameworks like AutoGen may not be universally suitable for all application requirements.
- **Performance Variability**: The effectiveness of frameworks can vary significantly based on project requirements, necessitating careful selection.

## Future Trends  
As AI technologies continue to break new ground, several trends are likely to shape the future of AI agent frameworks:
- **Increased Modularity**: Expect a rise in modular frameworks that allow developers to customize functionalities extensively.
- **Broader Integration Capabilities**: Future frameworks will likely integrate seamlessly with a broader range of tools and platforms in enterprise settings.
- **Enhanced Collaboration Features**: Advancements in inter-agent communication are anticipated, reinforcing collaboration both among agents and between humans and machines.

## Conclusion  
The AI agent framework landscape as of May 2025 is characterized by diversity and sophistication, with various frameworks tailored to meet specific enterprise needs. Understanding the strengths and weaknesses of each framework is essential for organizations looking to harness AI agents effectively. As developments continue, staying abreast of these changes will be crucial for leveraging the full potential of AI in highly competitive sectors.

## References  
1. Curotec. "Top AI Agent Frameworks: Insights from May 2025."  
2. AI21. "Evaluating AI Agent Frameworks for Enterprise Use."
3. Turing. "A Comprehensive Review of AI Agent Frameworks."
4. Langbase. "The State of AI Agents: Survey Results of 2025."
5. Vestra. "Top AI Agent Frameworks to Watch in 2025."  
6. Medium. "Best AI Agent Frameworks for Builders in 2025."  

---  
This report provides a holistic view of the AI agent framework landscape, appropriate for organizations seeking to understand current trends and make informed decisions regarding their AI implementations.

### As always, take a look at the trace

https://platform.openai.com/traces